In [46]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

In [47]:
# 自訂 function 可重新 reload
from importlib import reload
from utils import util_inference_result
reload(util_inference_result)
from utils.util_inference_result import *

In [83]:
# 4_mix
main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
result_folder = 'results/NYISO_4/NYISO_4_mix/T200_beta00.0001_betaT0.02/imputaiton_multiple_20k/'
dataset_folder = 'datasets/NYISO/dataset_4/mix/'
dataset_filename = 'load_array_mix_test.npy'

# 4_all
# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_4/NYISO_4_all/T200_beta00.0001_betaT0.02/imputaiton_multiple_20k/'
# dataset_folder = 'datasets/NYISO/dataset_4/all/'
# dataset_filename = 'load_array_all_test.npy'

# zone
# zone_name = ['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 
#              'LONGIL', 'MHK VL', 'MILLWD', 'N.Y.C.', 'NORTH', 
#              'WEST']
# zone_index = 2

# main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
# result_folder = 'results/NYISO_4/Zone/NYISO_4_'+ zone_name[zone_index] + '/T200_beta00.0001_betaT0.02/imputaiton_multiple_20k/'
# dataset_folder = 'datasets/NYISO/dataset_4/zone/'
# dataset_filename = zone_name[zone_index] + '_test.npy'



In [84]:
print(np.std(pred_data[0:10,0,:,0]))
print(np.mean(pred_data[0:10,0,:,0]))
print(np.median(pred_data[0:10,0,:,0]))


print(np.std(pred_data[:,0,:,0]))
print(np.mean(pred_data[:,0,:,0]))
print(np.median(pred_data[:,0,:,0]))

0.010573688
-1.1636393
-1.1682571
0.010573688
-1.1636393
-1.1682571


In [85]:
pred_data = read_multiple_imputations(main_folder + result_folder, 24)
print(pred_data.shape)

if len(pred_data.shape)==4: # 讀取 inference_multiples.py 的結果 len = 4; inference.py 則是 3
    pred_mean = np.mean(pred_data, axis = 0)
    pred_median = np.median(pred_data, axis = 0)
    print(pred_mean.shape)
else:    
    pred_data_target = pred_data[:,:,:,-24:].reshape(pred_data.shape[1],1,24)
    print(pred_data_target.shape)

file_path = os.path.join(main_folder + result_folder, 'original0.npy')
true_data = np.load(file_path)
print(true_data.shape)
true_data_target = true_data[:,:,-24:]
print(true_data_target.shape)

file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
true_data_o = np.swapaxes(np.load(file_path), axis1=1, axis2=2)
print(true_data_o.shape)

true_data_o_cond = true_data_o[:,:,0:168]
print(true_data_o_cond.shape)
true_data_o_target = true_data_o[:,:,-24:]
print(true_data_o_target.shape)

(5, 803, 1, 24)
(803, 1, 24)
(803, 1, 192)
(803, 1, 24)
(803, 1, 192)
(803, 1, 168)
(803, 1, 24)


In [86]:
# 因應使用 z normalization or range normalization，需使用不同的 resacle
if 'NYISO_4' in result_folder:
    print('NYISO_4')
    m = np.mean(true_data_o_cond, axis=2).reshape((pred_mean.shape[0], pred_mean.shape[1], 1))
    s = np.std(true_data_o_cond, axis=2).reshape((pred_mean.shape[0], pred_mean.shape[1], 1))

    pred_mean_o = pred_mean*s + m
    pred_median_o = pred_median*s + m

    print(f'median MAPE: {round(np.mean(abs((true_data_o_target - pred_median_o))/true_data_o_target)*100,2)}')
    print(f'MAPE: {round(np.mean(abs((true_data_o_target - pred_mean_o))/true_data_o_target)*100,4)}')
    print(f'MSE: {round(np.mean(np.square(true_data_target - pred_mean))*1,4)}')
    
elif 'NYISO_5' in result_folder:  
    print('NYISO_5')
    min_value = np.min(true_data_o_cond, axis=2).reshape((pred_mean.shape[0], pred_mean.shape[1], 1))
    max_value = np.max(true_data_o_cond, axis=2).reshape((pred_mean.shape[0], pred_mean.shape[1], 1))

    pred_mean_o = pred_mean*(max_value - min_value) + min_value
    pred_median_o = pred_median*(max_value - min_value) + min_value

    print(f'median MAPE: {round(np.mean(abs((true_data_o_target - pred_median_o))/true_data_o_target)*100,2)}')
    print(f'MAPE: {round(np.mean(abs((true_data_o_target - pred_mean_o))/true_data_o_target)*100,4)}')
    print(f'MSE: {round(np.mean(np.square(true_data_target - pred_mean))*1,4)}')

NYISO_4
median MAPE: 3.34
MAPE: 3.3948
MSE: 0.1584


In [69]:
zone_name = ['CAPITL', 'CENTRL', 'DUNWOOD', 'GENESE', 'HUD VL', 'LONGIL', 'MHK VL', 
             'MILLWD', 'N.Y.C.', 'NORTH', 'WEST']
if 'mix' in dataset_filename:
    print('mix')
    # mix 資料集看單一個 zone
    for i in range(11):
        num = int(true_data_o_target.shape[0]/11)
        zone_obs_num = range(num*(i), num*(i+1))
        print(f'{i} MAPE: {round(np.mean(abs((true_data_o_target[zone_obs_num,:,:] - pred_mean_o[zone_obs_num,:,:])/true_data_o_target[zone_obs_num,:,:]))*100,4)}')
        # print(f'MSE: {round(np.mean(np.square(true_data_target[:,i,:] - pred_mean[:,i,:]))*1,4)}')
if dataset_filename[:3]=='all' or dataset_filename[11:14]=='all':
    print('all')
    # all 資料集看單一個 zone
    for i in range(11):
        median_MAPE = round(np.mean(abs((true_data_o_target[:,i,:] - pred_median_o[:,i,:])/true_data_o_target[:,i,:]))*100,2)
        MAPE = round(np.mean(abs((true_data_o_target[:,i,:] - pred_mean_o[:,i,:])/true_data_o_target[:,i,:]))*100,4)
        MSE = round(np.mean(np.square(true_data_target[:,i,:] - pred_mean[:,i,:]))*1,4)
        print(f'{zone_name[i]}: median MAPE = {median_MAPE} MAPE = {MAPE}  MSE = {MSE}')
        

# Select which pkl

compare each imputation generated from different pkl

In [25]:
main_folder = '/home/hchuang/Documents/Project/SSSD_CP/src/'
result_folder = 'results/NYISO_mix/T200_beta00.0001_betaT0.02/compare_pkl_test_data_1/'
dataset_folder = 'datasets/NYISO/dataset_2/mix/'
dataset_filename = 'mix_test_data_1.npy'

pred_data = read_multiple_imputations(main_folder + result_folder, 24)

file_path = os.path.join(main_folder + result_folder, 'original0.npy')
true_data = np.load(file_path)
true_data_target = true_data[:,:,-24:]

file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
true_data_o = np.swapaxes(np.load(file_path), axis1=1, axis2=2)

true_data_o_cond = true_data_o[:,:,0:168]
true_data_o_target = true_data_o[:,:,-24:]

m = np.mean(true_data_o_cond, axis=2).reshape((true_data_o_cond.shape[0], 1, 1))
s = np.std(true_data_o_cond, axis=2).reshape((true_data_o_cond.shape[0], 1, 1))

# for i in range(17):
for i in [0,1,2,3,9,13,14,15,16]:    
    num = i+1
    file_name = str(num) + '0_imputation0.npy'
    file_path = os.path.join(main_folder + result_folder, file_name)
    pred_each = np.load(file_path)
    pred_each = pred_each[:,:,-24:]
    # pred_each = pred_data[i,:,:,:].reshape(pred_data.shape[1],pred_data.shape[2],pred_data.shape[3])

    pred_data_rescale = pred_each*s + m
    print(f'MAPE: {np.mean(abs((true_data_o_target - pred_data_rescale)/true_data_o_target))*100}')


MAPE: 3.5904037804097957
MAPE: 2.898665800129504
MAPE: 2.8213715364070935
MAPE: 2.744114768133877
MAPE: 2.634120765526516
MAPE: 2.580837563737816
MAPE: 2.5700620774130165
MAPE: 2.5964068438556995
MAPE: 2.5493624462224145


# mix data 對應到 zone


In [27]:
dataset_folder = 'datasets/NYISO/dataset_2/mix/'
dataset_filename = 'mix_test_data_0_stdd.npy'
file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
true_data = np.swapaxes(np.load(file_path), axis1=1, axis2=2)
print(true_data.shape)

(22, 1, 192)


In [38]:
dataset_folder = 'datasets/NYISO/dataset_2/zone/'
dataset_filename = 'WEST_test_data_0_stdd.npy'
file_path = os.path.join(main_folder + dataset_folder, dataset_filename)
zone_true_data = np.swapaxes(np.load(file_path), axis1=1, axis2=2)
print(zone_true_data.shape)

(2, 1, 192)


In [39]:
print(true_data[20:22,0,0])
print(zone_true_data[0:2,0,0])

[-0.84438167 -0.98628458]
[-0.84438167 -0.98628458]


In [12]:
2%1

0

In [24]:
n_iter = 0
while n_iter < 3:
    # print(n_iter)
    print(n_iter)
    for i in range(2):
        i+i
        # print(n_iter)
    n_iter += 1
    

0
1
2


10